<img src="https://drive.google.com/uc?export=view&id=1nMaBQ7g88duMECw_B4mH8ruxwnIwVzGP" width=300/> 

# ML-1: Standardization, Classification & Model evaluation
## Homework 3: Are you mad enough to sell more clothes?

**ML-1 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>

### Name of people who have worked on this homework:

## Table of Contents 
* [HW-3: Are you mad enough to sell more clothes?](#HW-3:-Are-you-mad-enough-to-sell-more-clothes?)
  * [Instructions](##Instructions)
  * [Overview](##Overview)
  * [Q1: Test & Training Sets & Standardization](##Q1:-Test-&-Training-Sets-&-Standardization)
    * [1.1 Train-Test Split](###1.1-Train-Test-Split)
    * [1.2 Standardizing data](##1.2-Standardizing-data)
  * [Q2: Classification with Lasso (L1) regularization](##Q2:-Classification-with-Lasso-(L1)-regularization)
    * [2.1 Baseline Classifiers](###2.1-Baseline-Classifiers)
    * [2.2 Classification Model - Logistic with Lasso (L1)](###2.2-Classification-Model-Logistic-with-Lasso-(L1))
    * [2.3 Prediction](###2.3-Prediction)
  * [Q3: Confusion Matrix, Costs, Averge Profit & Thresholds](##Q3:-Confusion-Matrix,-Costs,-Averge-Profit-&-Thresholds)
    * [3.1 Confusion Matrix](####3.1-Confusion-Matrix)
    * [3.2 Cost and Utility Matrix](####3.2-Cost-and-Utility-Matrix)
    * [3.3 Average Profit Per Person](####3.3-Average-Profit-Per-Person)
    * [3.4 Thresholds](####3.4-Thresholds)
  * [Q4: ROC Curves & Profit Curves](##Q4:-ROC-Curves-&-Profit-Curves)
    * [4.1 ROC Curves](####4.1-ROC-Curves)
    * [4.2 Profit Curves](####4.2-Profit-Curves)

## Instructions

- This homework should be submitted in pairs.

- Ensure you and your partner together have submitted the homework only once. Multiple submissions of the same work will be penalised and will cost you 2 points.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. 

- To work on the homework, you will first need to fork the repository into your GitHub account and clone it to work on it on your local computer. To submit your homework, push your homework into the same GitHub and upload the link on edStem.

- Submit the homework well before the given deadline. Submissions after the deadline will not be graded.

- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them.

- Comment your code well. This would help the graders in case there is any issue with the notebook while running. It is important to remember that the graders will not troubleshoot your code. 

- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 

- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```
- Your plots should include clear labels for the $x$ and $y$ axes as well as a descriptive title ("MSE plot" is not a descriptive title; "95 % confidence interval of coefficients of polynomial degree 5" is).

- **Ensure you make appropriate plots for all the questions it is applicable to, regardless of it being explicitly asked for.**

<hr style="height:2pt">

## Overview

The data set is from a fairly high end clothing chain store in the North East. You should be familiar with this data from Exercise 3 where you learnt about feature engineering.

You are a data analyst for this store. Your job is to write a report to the pointy-haired boss in which you show how you can increase the store's profit by targeting whom to send a catalog in the mail. Yes, you are in direct marketing. You are a quant amongst the "mad men". 

You need to explore and layout in simple terms, what the business needs to spend to increase its profit. In other words, you need a budget, and its your job to figure out how much as well.

We'll guide you through the process. There is much more you can explore, of-course, but this homework will walk you through an entire real world classification and analysis process with a finite amount of work and computer runtime.

You will

1. learn how to standardize the data
2. write a classifier on this data, including cross validation
3. learn how to compare this classifier to baseline classifiers that you better beat using a profit metric rather than an accuracy metric
4. understand and use prediction thresholds
5. understand the use a ROC curve
6. learn to use a profit curve to pick a model, thus directly reflecting the metric of importance

You have been provided with the cleaned data after numerous feature transformations.

The idea for this homework, and the attendant data set is taken from the book "Data Mining Methods and Models" by [Larose](http://www.dataminingconsultant.com/DMMM.htm).

---

In [5]:
# Run this cell to import the required libraries

%matplotlib inline
import numpy as np
import scipy as sp
from scipy.stats.stats import pearsonr
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import seaborn as sns
# sns.set_style("whitegrid")
# sns.set_context("poster")

sns.set_style()
sns.set_context("talk")

## Q1: Test & Training Sets & Standardization (15 points)

We standardize test and training sets separately. Specifically, we wish to standardize the non-indicator columns on both the test and training sets, by subtracting out the mean of the training set from the value, and dividing by the standard deviation of the training set. This helps us put all the continuous variables on the same scale.

#### 1.1 Train-Test Split (5 points)

**Split the data(`dftouse` dataframe) into train and test in 70:30 ratio. Print out the shape of your train and test features and labels. ('X_train','X_test','y_train','y_test')**

In [30]:
# loading data from csv file
df = pd.read_csv('./data/Cleaned_clothing_store_data.csv')
dftouse = df.copy()

In [1]:
# your code here

#### 1.2 Standardizing data (5 points)

Listed below are continuous features or variables that need to be standardized.

`PERCENT_VARS`: 15 variables providing the percentages spent by the customer on specific classes of clothing, including sweaters, knit tops, knit dresses, blouses, jackets, career pants, casual pants, shirts, dresses, suits, outerwear, jewelry, fashion, legwear, and the collectibles line(`P*`, `PJACKETS` for example)

`ZERO_IMPORTANT_VARS`: These are columns where the existence or lack thereof of a zero may be important in a classifier. We used our intuition to make these choices, believing that there is additional information encoded in having zeroes vs non-zeroes

`STANDARDIZABLE`: These are continuous columns(features) that need to be brought to the same scale. They include `PERCENT_VARS` and `ZERO_IMPORTANT_VARS` and a few other columns as well

In [32]:
# Run this cell to get the Standardizable features

PERCENT_VARS=[ u'PSWEATERS', u'PKNIT_TOPS', u'PKNIT_DRES', u'PBLOUSES', u'PJACKETS', u'PCAR_PNTS', u'PCAS_PNTS', u'PSHIRTS', 
              u'PDRESSES', u'PSUITS', u'POUTERWEAR', u'PJEWELRY', u'PFASHION', u'PLEGWEAR', u'PCOLLSPND']
ZERO_IMPORTANT_VARS = [u'PREVPD', u'AMSPEND', u'PSSPEND', u'CCSPEND', u'AXSPEND', u'RESPONDED', u'PERCRET']
STANDARDIZABLE = PERCENT_VARS + ZERO_IMPORTANT_VARS + [u'FRE', u'MON',  u'AVRG', u'GMP', u'PROMOS', u'DAYS', u'FREDAYS', u'MARKDOWN', u'CLASSES', u'COUPONS', u'STYLES',  u'MAILED',  u'RESPONSERATE', u'HI', u'LTFREDAY']

**1.2 a) Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDRARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.**

In [2]:
#your code here

**1.2 b) Should standardization be done on all the data or should it be done separately on the training & test sets? Explain with your reasoning**

*your answer here*

## Q2: Classification with Lasso (L1) regularization (30 points)

We will now take this data and write a classifier to predict the response, which is in the `RESP` column of `dftouse`. This response corresponds to asking the question: will a user targeted with our advertisement respond or not?

#### 2.1 Baseline Classifiers (4 points)

**2.1 a) What is the accuracy of a classifier that predicts that `no customer will respond to our mailing`? Why do you think the accuracy is the way it is?**

In [3]:
# your code here

**2.1 b) What is the accuracy of a classifier that predicts that `all customers will respond to our mailing`?**

In [4]:
# your code here

#### 2.2 Classification Model - Logistic with Lasso (L1) (21 points)

**2.2 a) Train a Logistic Regression Model with L1 regularization. Take all columns as features apart from the response variable `RESP`. Fine-tune your model by performing 5-fold CV (use the helper function below) to find the best value of the hyper-parameter 'C' from the follwing values: *'{0.001,0.01,0.1,1,10,100}'*.**

In [36]:
# helper function for cross-validation

def cv_optimize(clf, parameters, X, y, n_folds):

    gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_)
    best = gs.best_estimator_
    return best

In [37]:
# your code here

**2.2 b) The Lasso implements internally, a form of feature selection by setting many coefficients to zero. Print the top 10 non-zero coefficient features from the classifier sorted by the absolute magnitude of the coefficients (highest to lowest)**

In [5]:
# your code here

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. This can be done using the Pearson correlation coefficient between each of our features and the response.

The Pearson correlation method is the most common method to use for numerical variables; it assigns a value between − 1 and 1, where 0 is no correlation, 1 is total positive correlation, and − 1 is total negative correlation. This is interpreted as follows: a correlation value of 0.7 between two variables would indicate that a significant and positive relationship exists between the two. A positive correlation signifies that if variable A goes up, then B will also go up, whereas if the value of the correlation is negative, then if A increases, B decreases.

**2.2 c) Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation.**

**Is there much overlap with the feature selection performed by the LASSO?**

*Hint* : Use `pearsonr` from `scipy.stats.stats`. This has already been imported at the beginning

In [ ]:
# your code here

*your answer here*

**2.2 d) Compare the accuracy of this classifier with the `no customer responds baseline` model. Does this classifier seem worthwhile pursuing?**

*your answer here*



**2.2 e) Is accuracy really the best metric to evaluate the models above? Why/Why not?**

*your answer here*



#### 2.3 Prediction (5 points)

**Print out the accuracy on the train and test set separately**

In [1]:
# your code here

## Q3: Confusion Matrix, Costs, Averge Profit & Thresholds (35 points)

#### 3.1 Confusion Matrix (7 points)

The confusion matrix is of the following form:

![hwimages](./images/confusionmatrix.png)

**3.1 a) Plot the confusion matrix for the above models on the train & test data-sets using the helper function below(total 2 plots)**


In [44]:
# helper function to plot confusion matrix

def plot_confusion_matrix(model, features, labels, test=True):

    # Predict the values from the dataset
    y_pred = model.predict(features)
    # Convert validation observations to one hot vectors
    y_true = labels 
    # compute the confusion matrix
    confusion_mtx = confusion_matrix(y_true, y_pred) 

    df_cm = pd.DataFrame(confusion_mtx, range(2),
                      range(2))
    sns.heatmap(df_cm, annot=True, annot_kws = {'size':15}, cmap = 'Blues',fmt = 'd',
                norm=LogNorm(df_cm.values.min(),df_cm.values.max()),
                cbar_kws={"ticks":[0,1,10,1e2,1e3,1e4]},vmin=0.001, vmax=10000)
    plt.tight_layout()
    if test:
        plt.title('Confusion matrix on Test data')
    else:
        plt.title('Confusion matrix on Train data')
    plt.ylabel('Observed label')
    plt.xlabel('Predicted label')
    plt.ylim(2,0)
    plt.show()

In [7]:
# your code here

**3.1 b) Calculate and print out the Observed Negatives(ON), Observed Positives(OP), Predicted Negatives(PN) & Predicted Positives(PP) for the test data-set**

In [8]:
# your code here

In addition to these four quantities, the confusion matrix gives us more details on proper classifications and mis-classifications from our classifier:

- the samples that are +ive and the classifier predicts as +ive are called True Positives (TP). These are folks we correctly identified as responders,and thus sending them a mailing would result in a sale for us. True Positives are great. We do incur the cost of mailing them, but we like to because they will come into the store to buy.
- the samples that are -ive and the classifier predicts (wrongly) as +ive are called False Positives (FP). False Positives incur us the cost of mailing them as well, but are not very costly. These are people who wouldnt have responded, but we sent them a mailing because our classifier mispredicted them as buyers. Thus, for them, we only incur the cost of preparing the mailing and mailing it to them.
- the samples that are -ive and the classifier predicts as -ive are called True Negatives (TN). These are folks we correctly identified as not-responding, and thus we dont waste any money on sending them a mailing. This is a great classification for us.
- the samples that are +ive and the classifier predicts as -ive are called False Negatives (FN). False negatives are VERY costly: these are folks who would have responded to us had we mailed them, but we didnt target them, leading to huge lost sales per person. Notice that our Logistic classifier has tons of False Negatives

It is not enough to simply identify these categories from the confusion matrix. Rather, we want to sit down with our business team and identify the costs associated with each of the 4 classification situations above. Keep in mind that these costs might even change from year to year or even more suddenly: this is why it is important to have marketing and sales people on your data science teams. (See Patil, D. J. Building data science teams. " O'Reilly Media, Inc.", 2011.
 for more details).

#### 3.2 Cost  and Utility Matrix (6 points)

We use the costs to write a **risk or cost matrix** in the same form as the confusion matrix above. 

![cost matrix](images/costmatrix.png)

The negative of the cost matrix is called the **utility matrix or profit matrix** `u`. We shall use this in the next part of the homework.

Lets assume the amortized cost of mailing it is \$3. Lets assume additionally that the profit margin on a sale is 30% (we are a high end clothing chain).
`True Negatives (TN)` cost us nothing but gain us nothing either. 

In [47]:
# true negative cost
tnc = 0.0

From the average cost of a sale, and the 30% profit assumption, we calculate `tpc`, the cost of a `True Positive (TP)`. Note: `tpc` must be negative, since we are talking about costs.

The `tpc` takes into account the cost of mailing to the respondent & the cost of the coupon, and since our mailing works, we subtract out the profit. We use the average of the `AVRG` column(which is the average money spent by a customer on each visit) multiplied by the profit margin as the profit per person.

In [55]:
# true positive cost
mail = 3 
profit_margin = 0.3
tpc = round(mail - df.AVRG.mean()*profit_margin,2)

The `False Negative (FN)` is a lost sale for us! We didnt mail them, and they didnt spend the money. They would have if we mailed them. So we lost a certain profit per such false negative! Thus the false-negative cost, given by `fnc`, is:

In [56]:
fnc = round(np.mean(df.AVRG)*profit_margin,2)

This leaves us with `False Positives (FP)`. This is a person who would not have responded but you wasted $3 on. So the false positive cost, (`fpc`) is:

In [57]:
fpc = mail

In [58]:
print('TNC:', tnc,'\n','TPC:', tpc,'\n','FNC:', fnc,'\n','FPC:', fpc)

TNC: 0.0 
 TPC: -31.17 
 FNC: 34.17 
 FPC: 3


**Print out the profit matrix array. Use the costs described above**

In [9]:
# your code here

#### 3.3 Average Profit Per Person (16 points)

The Average Profit Per Person can be found by multiplying the utility matrix (u) by the confusion matrix elementwise, and dividing by the sum of the elements in the confusion matrix, or the test set size.

**3.3 a) Write a function to calculate the average profit per person by following the instructions below**

In [10]:
def average_profit_pp(y, ypred, u):
    """
    Function
    --------
    average_profit_pp

    Inputs
    ------
    y: the observed response value (true value)
    ypred: the predicted response value
    u: Utility or the Profit matrix

    Returns
    -------
    score: average profit per person calculated
    """
    # your code here
    

**3.3 b) Calculate and Print the Avg Profit Per Person on our test data for the 2 baseline classifiers:** 
- Dont Send to Anyone Baseline Classifier (predicts that no customer will respond)
- Send to Everyone Baseline Classifier (predicts that all customers will respond)

*your answer here*

**3.3 c) Taking the average profit per person as the comparison metric, which one of the above 2 classifiers is the one to beat?**

*your answer here*



**3.3 d) What is the Avg Profit Per Person for our Logistic-L1 classifier?**

*your answer here*

#### 3.4 Thresholds (6 points)

The `sklearn` API function `predict` assumes a threshold probability of having a +ve sample to be 0.5; that is, if a sample has a greater than 0.5 chance of being a 1, assume it is so. In the predictions above, we've done exactly that - if probability is >= 0.5, then `ypred` is 1 else 0.

By varying the threshold, the `ypred` and consequently the confusion matrix and the avg profit per person can change. 

**3.4 a) The thresholds are NOT always kept at 0.5 while solving a classification problem. Why do you think this happens? Can you think of any other real world example where we would want to change the threshold from 0.5?**

*your answer here*



**3.4 b) Find out the average profit per person for a given thresholds of 0.05. You can use the helper function provided below to help with this problem**

In [71]:
# helper function to return the predictions for a given threshold 't'

def t_repredict(clf,t,xtest):
    probs=clf.predict_proba(xtest)
    p1 = probs[:,1]
    ypred = (p1 > t)*1
    return ypred

In [2]:
# your code here

**3.4 c) Compare the above classifier with our baseline classifiers. What do you observe? Is this classifier worth pursuing?**

*your answer here*



## Part 4: ROC Curves & Profit Curves (20 points)

#### 4.1 ROC Curves (3 points)

ROC curves are a set of classifiers, many of them, each point corresponding to a different threshold. They are useful to compare classifiers to each other and also to baseline models.
(In the standard scenario, where we used the  classifier accuracy, this threshold is implicitly set at 0.5, and we have only one point on the ROC curve.).

The practical way to do this is to order the samples by probability of being positive. Then consider the sample with the highest score or highest probability of being positive. At first, only this sample is positive. Then, we take the sample with the next highest score, and call it positive. As we go down the list, we go down a threshold in score or probability. 

Now, for each such situation: only 1 positive, now 2 positive,....you can imagine a different classifier with a different confusion matrix. It will have its own false positives, three positives, etc. Its actually the same original classifier, but with a different threshold each time.

As we keep going down the list, decreasing the threshold, more and more samples become positive, and at first, the true positives rise faster than the false positives. Once past a certain point, false positives increase faster than true positives. Now, if you want a balanced classifier, you look at this turn-around point. But if you want a classifier which penalizes false positives and false negatives differently, the point you want is different.

To make a ROC curve you plot the True Positive Rate, 

$$TPR=\frac{TP}{OP}$$

against the False Positive Rate,

$$FPR=\frac{FP}{ON}$$

as you go through this process of going down the list of samples. ROC curves are useful because they calculate one classifier per threshold and show you where you are in TPR/FPR space without making any assumptions about the utility matrix or which threshold is appropriate.

A rote reading of the ROC curve (go to the "northwest" corner) is a bad idea: you must fold in the curve with any assumptions you are making about the utilities. In our case we have both an asymmetric data set, and asymmetric risk, so the north west corner may not be the right spot. Still, on the whole, a curve with a greater AUC (area under curve), or further away from the line of randomness, will give us a rough idea of what might be a better classifier.

**Plot the ROC curves for the Logistic-L1 model. Use the helper function below given below to help you out with this**

In [79]:
# helper function to plot the ROC Curve

def make_roc(clf, ytest, xtest):
    
    fig,ax=plt.subplots(figsize=(12,8))
    fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, '.-', alpha=0.3, label='ROC curve (area = %0.2f)' % (roc_auc))
    
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
    )
    for k in range(0, fpr.shape[0],200):
        #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
        threshold = str(np.round(thresholds[k], 2))
        ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
        
    ax.plot([0, 1], [0, 1], 'k--', label='No Model')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC')
    ax.legend(loc="lower right")
    
    return ax
        
    

In [3]:
#your code here

#### 4.2 Profit Curves (17 points)

Just like in a ROC curve, we go down the sorted (by score or probability) list of samples. We one-by-one add an additional sample to our positive samples, noting down the classifier's TPR and FPR and threshold. In addition to what we do for the ROC curve, we now also note down the percentage of our list of samples predicted as positive. Remember we start from the most positive, where the percentage labelled as positive would be minuscule, like 0.1 or so and the threshold like a 0.99 in probability or so. As we decrease the threshold, the percentage predicted to be positive clearly increases until everything is predicted positive at a threshold of 0. What we now do is, at each such additional sample/threshold (given to us by the `roc_curve` function from `sklearn`), we calculate the expected profit per person and plot it against the percentage predicted positive by that threshold to produce a profit curve. Thus, small percentages correspond to samples most likely to be positive: a percentage of 8% means the top 8% of our samples ranked by likelihood of being positive.


**4.2 a) Write a function to calculate the percentage of samples classified/predicted as positive given the TPR and FPR of a classifier. Follow the instructions below**

In [15]:
def percentage(tpr, fpr, priorp, priorn):
    """
    Function
    --------
    percentage

    Inputs
    ------
    tpr: True Positive Rate
    fpr: False Positive Rate
    priorp: the probability of observed +ives (OP) on our test set
    priorn: the probability of observed +ives (ON) on our test set

    Returns
    -------
    perc: percentage of samples classified as positive
    """
    #your code here
    
    

**4.2 b) Write a function to calculate the Expected Profit Per Person given the TPR & FPR from a classifier(this is different than our `average_profit_pp` above as we now want this in terms of TPR and FPR.**

In [16]:
def av_profit(tpr, fpr, util, priorp, priorn):
    """
    Function
    --------
    percentage

    Inputs
    ------
    tpr: True Positive Rate
    fpr: False Positive Rate
    util: Utility Matrix
    priorp: the probability of observed +ives (OP) on our test set
    priorn: the probability of observed +ives (ON) on our test set

    Returns
    -------
    profit: the average profit per person at this (fpr, tpr) point in this ROC space.
    
    Notes
    -----
    see make_profit below for an example of how this is used
    """
    #your code here
    
    

**4.2 c) Plot the Profit Curve for our Logistic-L1 model using the helper function below. Also plot the profit lines for the STE(Send to Everyone) and the DSTE (Dont Send to Anyone) baseline classifiers. Make sure the labels are appropriately named and the legend is clearly displayed**

In [101]:
# helper function to plot the profit curve

def make_profit(clf, ytest, xtest, util):
    
    fig,ax=plt.subplots(figsize=(12,8))
    fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    priorp=np.mean(ytest)
    priorn=1. - priorp
    ben=[]
    percs=[]
    
    for i,t in enumerate(thresholds):
        perc=percentage(tpr[i], fpr[i], priorp, priorn)
        ev = av_profit(tpr[i], fpr[i], util, priorp, priorn)
        ben.append(ev)
        percs.append(perc*100)
    ax.plot(percs, ben, '-', alpha=0.3, markersize=5, label='Profit Curve for model' )
   
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
    boxstyle='round,pad=0.3', alpha=0.2,
    )
    for k in range(0, fpr.shape[0],200):
        #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
        threshold = str(np.round(thresholds[k], 2))
        ax.annotate(threshold, (percs[k], ben[k]), **label_kwargs)
                
    ax.set_xlabel('% Predicted Positive')
    ax.set_ylabel('Avg Profit Per Person')
    ax.set_title('Profit Curve')
    return ax


In [4]:
# your code here

**4.2 d) Why you only start making a profit at a certain percentage?**

*your answer here*



**4.2 e) What is the region on interest on the graph that you might want to communicate with the company managers in terms of maximizing your profit over the baseline?**

*your answer here*

